<a href="https://colab.research.google.com/github/lilasch/Facial_Attribute_CNN/blob/main/basicmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Facial Attribute Classification**

Lila Schisgal and Phoebe Jeske


First, we import our data from two text files and partition it into training and validation data.

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
import tensorflow
import pandas as pd
import cv2
import numpy as np
import glob

# Load attributes csv
attributes = pd.read_csv("list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")

# Load csv with partitions values
partitions = pd.read_csv("list_eval_partition.txt", delimiter="\s+|\t", header = None)

partitions.columns = ['image_name', 'dataset'] # setting column header names for partitions
attributes['dataset'] = partitions['dataset'].values # copying the partition values into the attributes df


Mounted at /content/drive


<ipython-input-53-c78c5ea5c41d>:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  attributes = pd.read_csv("/content/drive/MyDrive/list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")
<ipython-input-53-c78c5ea5c41d>:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  partitions = pd.read_csv("/content/drive/MyDrive/list_eval_partition.txt", delimiter="\s+|\t", header = None)


We unzip the file with our images.

In [ ]:
# unzipping image data
import zipfile
zippath = 'img_align_celeba.zip'
targetfolder = 'all_images'

with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(targetfolder)

For now, we're just testing whether our network can identify whether someone is wearing glasses.

In [ ]:
# filter so just the eyeglasses attribute is included
glasses_df = attributes.iloc[:, [15, 40]] # 40 attributes

In [ ]:
# the image filenames need to be in their own column
glasses_df = glasses_df.reset_index()

We split our dataframes into subdataframes for train, validation, and test.

In [ ]:
# rename columns
glasses_df.rename(columns={'index': 'image_names'}, inplace=True)
glasses_df.rename(columns={'Eyeglasses': 'eyeglasses_or_not'}, inplace=True)

# splitting the data
train = glasses_df[glasses_df['dataset'] == 0]
train.drop(columns='dataset', inplace=True)
valid = glasses_df[glasses_df['dataset'] == 1]
valid.drop(columns='dataset', inplace=True)
test = glasses_df[glasses_df['dataset'] == 2]
test.drop(columns='dataset', inplace=True)

<ipython-input-56-fa326fac323d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(columns='dataset', inplace=True)
<ipython-input-56-fa326fac323d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid.drop(columns='dataset', inplace=True)
<ipython-input-56-fa326fac323d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns='dataset', inplace=True)


"\ntrain_labels = train.loc[:,'eyeglasses_or_not']\nvalid_labels = valid.loc[:,'eyeglasses_or_not']\ntest_labels = test.loc[:,'eyeglasses_or_not']\ny_train=keras.utils.to_categorical(train_labels)\ny_valid=keras.utils.to_categorical(valid_labels)\ny_test=keras.utils.to_categorical(test_labels)"

We use an ImageDataGenerator to normalize and load our images from each dataframe.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator() # ImageDataGenerator for train
valid_datagen = ImageDataGenerator() # ImageDataGenerator for valid

train["eyeglasses_or_not"] = train["eyeglasses_or_not"].astype('str') # requires target in string format
valid["eyeglasses_or_not"] = valid["eyeglasses_or_not"].astype('str')

train_generator = train_datagen.flow_from_dataframe(dataframe=train,
                                              directory='all_images/img_align_celeba/',
                                              x_col="image_names",
                                              y_col="eyeglasses_or_not",
                                              subset="training",
                                              class_mode="binary",
                                              batch_size=64,
                                              rescale=1.0/255,
                                              target_size=(109,89)
                                              )

valid_generator = valid_datagen.flow_from_dataframe(dataframe=valid,
                                              directory='all_images/img_align_celeba/',
                                              x_col="image_names",
                                              y_col="eyeglasses_or_not",
                                              class_mode="binary",
                                              batch_size=64,
                                              rescale=1.0/255,
                                              target_size=(109,89)
                                              )

<ipython-input-61-6f77becb2fd9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["eyeglasses_or_not"] = train["eyeglasses_or_not"].astype('str') # requires target in string format
<ipython-input-61-6f77becb2fd9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid["eyeglasses_or_not"] = valid["eyeglasses_or_not"].astype('str')


Found 162770 validated image filenames belonging to 2 classes.
Found 19867 validated image filenames belonging to 2 classes.


We define a model with two convolutional layers and two dense layers with a binary output (probability that they are wearing glasses).

In [ ]:
model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                 input_shape=(109, 89, 3)))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Flatten())
model.add(Dense(units=32, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 109, 89, 75)       2100      
                                                                 
 conv2d_24 (Conv2D)          (None, 109, 89, 50)       33800     
                                                                 
 flatten_15 (Flatten)        (None, 485050)            0         
                                                                 
 dense_33 (Dense)            (None, 32)                15521632  
                                                                 
 dense_34 (Dense)            (None, 1)                 33        
                                                                 
Total params: 15557565 (59.35 MB)
Trainable params: 15557565 (59.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


We train our model on the training data.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(generator=train_generator,
          epochs=20,
          steps_per_epoch=train.shape[0]//64,
          validation_data=valid_generator,
          validation_steps=valid.shape[0]//64,
          verbose=1)

<ipython-input-63-10047e0ea62a>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/20
1110/2543 [============>.................] - ETA: 1:17:21 - loss: 1.4382 - accuracy: 0.9346